In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import eval_extraction
import openai
import extraction
openai.api_key = open('/home/chansingh/.OPENAI_KEY').read().strip()

# Load the data

In [ ]:
df = pd.read_pickle("../data/data_clean.pkl")[
    [
        "id",
        "paper___url",
        "paper___raw_text",
        "paper___year",
        "participants___male",
        "participants___female",
        "participants___total",
        "participants___white",
        "participants___black",
        "participants___latino",
        "participants___asian",
    ]
]

In [ ]:
df.tail(n=300)

# Make predictions

Current outputs:
- num_male, num_female, num_total, num_male_evidence_span, num_female_evidence_span, num_total_evidence_span
- num_white, num_black, num_latino, num_asian, race_evidence_span

Targets:
- 'participants___male', 'participants___female', 'participants___total'
- 'participants___white', 'participants___black', 'participants___latino', 'participants___asian'

In [ ]:
idxs = df['paper___raw_text'].notna()
texts = df[idxs]['paper___raw_text'].tolist()
extractions = extraction.extract_nums_df(texts)
for k in extractions.keys():
    df.loc[idxs, k] = extractions[k]

### Evaluate
Evaluates whether each extracted number is within 1 of the human-labeled value

In [ ]:
# Apply some postprocessing to the predictions (convert percentages)
gender_count_processed = df.apply(eval_extraction.process_gender_counts, axis=1)
df["num_male"] = gender_count_processed.apply(lambda x: x[0]).astype(str)
df["num_female"] = gender_count_processed.apply(lambda x: x[1]).astype(str)
# print("Total n (with paper text)", idxs.sum())
eval_extraction.compute_metrics_within_1(
    df,
    preds_col_to_gt_col_dict={
        "num_male": "participants___male",
        "num_female": "participants___female",
        "num_total": "participants___total",
    },
)